In [1]:
import nltk
nltk.download('words')
from nltk.corpus import words
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
from nltk.tokenize import MWETokenizer, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from collections import Counter

In [3]:
import numpy as np
import tqdm
import pandas as pd
import pathlib
import os
import re
import string
import time
from random import random
from collections import defaultdict
from typing import List, Union
import pprint
import json
import tensorflow as tf
from gensim.models import Word2Vec
from time import time

In [4]:
from sklearn.cluster import KMeans

#### Mount to Google Drive

In [5]:
from google.colab import drive
drive.mount('/content/drive')

product_reviews_dir = 'product_reviews'
drive_path = "/content/drive" + "/My Drive/Colab Notebooks/" + product_reviews_dir
files = os.listdir(drive_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
def read_data(path: str) -> List[str]:
    """
        Read files from a directory and then append the data of each file into a list.
    :param path:
    :return:
    """
    files = os.listdir(path)  # to be iterated over in folder
    content = ""  # to store the contents of each file in folder
    files.remove('README.txt')
    for filename in files:
        file_path = f'{path}/{filename}'
        file_extension = pathlib.Path(file_path).suffix
        if '.txt' == file_extension:  # ensure we only read from *.txt files
            with open(file_path, encoding='utf8') as f_open:
                content_lines = f_open.read()  # grab the contents of the file
            content += content_lines
    return content

In [7]:
content = read_data(drive_path)
print(content[:100])

[t]
router[+2]##This router does everything that it is supposed to do, so i dont really know how to 


In [8]:
def clean_document(document: str) -> str:
    """
        Perform text and character removal
    """
    # Normalize text by converting to lower case to minimise query time, computational and memory costs
    document = document.lower()
    # Remove redundant characters directly with custom regex to ease tokenization
    reg_to_drop = r'''(?x)           # flag to allow comments and multi-line regex
              | \( | \)              # remove ()-brackets as their contents can be useful
              | ^\$[\d,]+(\.\d*)?$   # cut out the money expressions
              | [^(!"#$%&'()*+, -.\/:;<=>?@[\]^_`{|}~) | (a-zA-Z0-9\s)]  # remove non-word, non-digit, non-punctuation characters, non-space, e.g. '•', '≠'
              | \\[u|U][a-zA-Z0-9]+  # remove codes like `\u00F4`
    '''
    reg_compiler = re.compile(reg_to_drop)
    document = reg_compiler.sub(r'', document)

    # Handle apostrophes
    document = re.sub(r"won\'t", "will not", document)
    document = re.sub(r"n\'t", " not", document)
    document = re.sub(r"\'t", " not", document)
    document = re.sub(r"\'m", " am", document)
    document = re.sub(r"\'re", " are", document)
    document = re.sub(r"\'ll", " will", document)
    document = re.sub(r"\'ve", " have", document)

    # Add extra spaces to \n and \#
    document = re.sub(r"\n", "\n ", document)
    document = re.sub(r"##", "## ", document)
    document = re.sub(r"\.\.\.", " ", document)

    # Handle shortenings
    document = re.sub(r" u ", " you ", document)
    # Transform sentiments
    document = re.sub(r"\[u\]", "", document)
    document = re.sub(r"\[p\]", "", document)
    document = re.sub(r"\[s\]", "", document)
    document = re.sub(r"\[cc\]", "", document)
    document = re.sub(r"\[cs\]", "", document)

    return document

In [9]:
print(content[:250])
print()
print(clean_document(content[:250]))

[t]
router[+2]##This router does everything that it is supposed to do, so i dont really know how to talk that bad about it. 
setup[+2], installation[+2] ##It was a very quick setup and installation, in fact the disc that it comes with pretty much mak

[t]
 router[+2]## this router does everything that it is supposed to do, so i dont really know how to talk that bad about it. 
 setup[+2], installation[+2] ## it was a very quick setup and installation, in fact the disc that it comes with pretty much mak


In [10]:
def process_document(document: str, drop_punct: bool = True, drop_digits: bool = True):
    """
      Pre-process a document and return a list of its terms
    :param document:
    :return:
    """
    # Clean the data
    document = clean_document(document)

    # Apply NLTK’s recommended word tokenizer for single word expressions (swe)
    word_tokens = word_tokenize(document)

    if drop_punct:
        punc = r'''[\!\(\)\-\[\]\{\}\;\:\'\"\,\<\>\.\/\?\@\$\%\\\^\&\*\_\~\`]'''
        word_tokens = [w for w in word_tokens if not re.search(punc, w)]
        # word_tokens = [w for w in word_tokens if not w in ['.', ',', '#', 'l', '[', ']']]
    
    if drop_digits:
        digit = r'''[0-9]+'''
        word_tokens = [w for w in word_tokens if not re.search(digit, w)]

    # Remove STOP words
    stop_words = set(stopwords.words('english'))
    filtered_data = [w for w in word_tokens if not w.lower() in stop_words]

    stemmer = SnowballStemmer(language='english')
    filtered_data = [stemmer.stem(w) for w in filtered_data]

    return filtered_data

In [11]:
swe_list = process_document(content)

In [12]:
sorted_words = Counter(swe_list)

In [13]:
print(sorted_words.most_common(51))

[('#', 8543), ('use', 592), ('phone', 348), ('router', 336), ('one', 331), ('get', 331), ('ipod', 328), ('camera', 316), ('player', 311), ('work', 281), ('batteri', 261), ('diaper', 230), ('like', 218), ('product', 214), ('great', 194), ('time', 184), ('featur', 182), ('problem', 179), ('good', 176), ('would', 173), ('look', 171), ('zen', 171), ('qualiti', 170), ('instal', 156), ('also', 156), ('sound', 155), ('take', 150), ('need', 149), ('softwar', 147), ('comput', 145), ('pictur', 140), ('want', 138), ('realli', 136), ('micro', 134), ('go', 129), ('well', 127), ('even', 124), ('buy', 123), ('thing', 123), ('easi', 123), ('first', 119), ('creativ', 119), ('review', 117), ('much', 115), ('make', 115), ('bag', 114), ('better', 113), ('size', 113), ('champ', 112), ('connect', 104), ('set', 103)]


In [14]:
top50_words = [w for w, v in dict(sorted_words.most_common(51)).items() if len(w) > 1]
assert len(top50_words) == 50, f'Size is {len(top50_words)}'

In [15]:
print(top50_words)

['use', 'phone', 'router', 'one', 'get', 'ipod', 'camera', 'player', 'work', 'batteri', 'diaper', 'like', 'product', 'great', 'time', 'featur', 'problem', 'good', 'would', 'look', 'zen', 'qualiti', 'instal', 'also', 'sound', 'take', 'need', 'softwar', 'comput', 'pictur', 'want', 'realli', 'micro', 'go', 'well', 'even', 'buy', 'thing', 'easi', 'first', 'creativ', 'review', 'much', 'make', 'bag', 'better', 'size', 'champ', 'connect', 'set']


In [16]:
print(swe_list)

['router', '#', '#', 'router', 'everyth', 'suppos', 'dont', 'realli', 'know', 'talk', 'bad', 'setup', 'instal', '#', '#', 'quick', 'setup', 'instal', 'fact', 'disc', 'come', 'pretti', 'much', 'make', 'sure', 'cant', 'mess', 'instal', '#', '#', 'mean', 'tech', 'junki', 'abl', 'instal', 'abl', 'put', 'cd', 'comput', 'tell', 'work', '#', '#', 'work', 'great', 'usual', 'full', 'mbps', 'although', 'everi', 'drop', 'around', 'mbps', 'floor', 'router', '#', '#', 'happen', 'everi', 'often', 'big', 'drawback', 'realli', 'littl', 'slower', 'usual', 'router', '#', '#', 'realli', 'great', 'buy', 'lookin', 'one', 'modem', 'mani', 'comput', 'around', 'hous', 'router', '#', '#', 'comput', 'hous', 'get', 'wireless', 'connect', 'router', 'everybodi', 'happi', '#', '#', 'realli', 'know', 'peopl', 'tear', 'router', '#', '#', 'apart', 'review', 'talk', 'instal', 'problem', 'setup', 'router', '+', '#', '#', 'easiest', 'thing', 'setup', 'thought', 'said', 'buy', 'router', 'instal', 'router', '#', '#', 'prob

#### Iterate over the lines, preprocess it and recreate a new sentence-level list

In [17]:
swe_list_reversed = []
top_word_stats = {w:0 for w in top50_words}
top_word_reverse_stats = {w[::-1]:0 for w in top50_words}

# iterate over the preprocessed document tokens
for token in swe_list:
    if token in top50_words:
        # randomly revert top words per occurrence
        if random() < 0.5:
            # reverse randomly selected top word
            swe_list_reversed.append(token[::-1])
            # record stats
            top_word_stats[token] += 1
            top_word_reverse_stats[token[::-1]] += 1
        else:
            # keep selected top word as it is
            swe_list_reversed.append(token)
            # record stats
            top_word_stats[token] += 1
    else:
        swe_list_reversed.append(token)

# ensure the generated reverse list is the same size as the original one
assert len(swe_list_reversed) == len(swe_list)

print(swe_list_reversed)
print(top_word_stats)
print(top_word_reverse_stats)

['router', '#', '#', 'router', 'everyth', 'suppos', 'dont', 'illaer', 'know', 'talk', 'bad', 'setup', 'instal', '#', '#', 'quick', 'setup', 'latsni', 'fact', 'disc', 'come', 'pretti', 'hcum', 'ekam', 'sure', 'cant', 'mess', 'latsni', '#', '#', 'mean', 'tech', 'junki', 'abl', 'instal', 'abl', 'put', 'cd', 'comput', 'tell', 'work', '#', '#', 'krow', 'great', 'usual', 'full', 'mbps', 'although', 'everi', 'drop', 'around', 'mbps', 'floor', 'router', '#', '#', 'happen', 'everi', 'often', 'big', 'drawback', 'realli', 'littl', 'slower', 'usual', 'router', '#', '#', 'illaer', 'great', 'buy', 'lookin', 'eno', 'modem', 'mani', 'tupmoc', 'around', 'hous', 'retuor', '#', '#', 'comput', 'hous', 'get', 'wireless', 'tcennoc', 'router', 'everybodi', 'happi', '#', '#', 'realli', 'know', 'peopl', 'tear', 'retuor', '#', '#', 'apart', 'review', 'talk', 'instal', 'melborp', 'setup', 'retuor', '+', '#', '#', 'easiest', 'gniht', 'setup', 'thought', 'said', 'buy', 'retuor', 'latsni', 'retuor', '#', '#', 'prob

In [18]:
# Create a sentence-level list for word2vec input
sentences = []
sent = []
for swe_entry in swe_list_reversed:
    if swe_entry == '#':
        # check it sentence has words (e.g., `#`)
        if sent:
            sentences.append(sent)
            # print(sent)
            sent = []
    else:
        sent.append(swe_entry)
print(sentences[:5])
print(len(sentences))

[['router'], ['router', 'everyth', 'suppos', 'dont', 'illaer', 'know', 'talk', 'bad', 'setup', 'instal'], ['quick', 'setup', 'latsni', 'fact', 'disc', 'come', 'pretti', 'hcum', 'ekam', 'sure', 'cant', 'mess', 'latsni'], ['mean', 'tech', 'junki', 'abl', 'instal', 'abl', 'put', 'cd', 'comput', 'tell', 'work'], ['krow', 'great', 'usual', 'full', 'mbps', 'although', 'everi', 'drop', 'around', 'mbps', 'floor', 'router']]
4258


#### Create feature matrix

In [19]:
word_labels = list(top_word_stats.keys()) + list(top_word_reverse_stats.keys())

In [20]:
def create_feature_matrix(w2v_model, verbose: bool = False):
    # Combine top 50 words with their reversed versions
    word_matrix = np.array([])

    # create a feature matrix
    for w in word_labels:
        if not len(word_matrix):
            word_matrix = w2v_model[w]
        else:  
            word_matrix = np.vstack([word_matrix, w2v_model[w]])

    # Ensure dimensions are consistent
    assert word_matrix.shape == (len(word_labels), d)
    if verbose:
        print(word_matrix.shape)
    return word_matrix

#### Calculate accuracy score

In [21]:
def get_word_index(word: str):
    return y.index(word)

In [22]:
def calculate_accuracy(X, kmeans, verbose: bool = False):
    # Calculate accuracy
    verbose = False
    correctness = 0
    perc_correct = 0

    for w in top50_words:
        # get array entries containing only a word and its reverse
        entry = X[[get_word_index(w), get_word_index(w[::-1])]]
        out = kmeans.predict(entry)
        if out[0] == out[1]:
            correctness += 1
        if verbose:
            print(w, w[::-1], end=" -> ")  
            print(out)

    perc_correct = np.round(correctness / n_clusters, 4) * 100
    if verbose:
        print(f'Correct pairs: {perc_correct}%')
    return perc_correct

Hyperparameter tuning

In [23]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [24]:
# fixed params
y = word_labels  # list(top_word_stats.keys()) + list(top_word_reverse_stats.keys())
n_clusters = int(len(y)/2) # 50

# hyperparameter tuning stats
hyperparameter_stats = []

# pipeline
for epochs in tqdm.tqdm([5, 10, 30, 50, 75, 100]):
    for d in [100, 164, 200, 300]:
        for window in [2, 3, 4, 5]:
            for sg in [0, 1]:
                for i in range(10):
                    # keep track of time
                    start = time()

                    # Create word2vec model
                    w2v_model = Word2Vec(window=window,
                                        size=d,
                                        sg=sg)
                    w2v_model.build_vocab(sentences)
                    w2v_model.train(sentences, 
                                    total_examples=w2v_model.corpus_count, 
                                    epochs=epochs)
                    w2v_model.init_sims(replace=True)

                    # transform the model vectors to a single matrix
                    X = create_feature_matrix(w2v_model)

                    # Apply clustering on the feature matrix
                    # The "elkan" variation can be more efficient on some datasets 
                    # with well-defined clusters, by using the triangle inequality
                    kmeans = KMeans(n_clusters, algorithm='elkan').fit(X, y)

                    end = time()

                    # Calculate accuracy score
                    accuracy = calculate_accuracy(X, kmeans)
                    hyperparameter_entry = {
                        'epochs': epochs,
                        'n_features': d,
                        'window': window,
                        'word2vec': "cbow" if sg == 0 else "skip-gram",
                        'iteration': i,
                        'accuracy': np.round(accuracy, 2),
                        'time (s)': np.round(end-start, 2),
                        'empty_clusters': n_clusters - len(set(kmeans.labels_))
                    }
                    print(hyperparameter_entry)
                    hyperparameter_stats.append(hyperparameter_entry)

  0%|          | 0/6 [00:00<?, ?it/s]WARNING:gensim.models.base_any2vec:under 10 jobs per worker: consider setting a smaller `batch_words' for smoother alpha decay


{'epochs': 5, 'n_features': 100, 'window': 2, 'word2vec': 'cbow', 'iteration': 0, 'accuracy': 12.0, 'time (s)': 1.11, 'empty_clusters': 0}


{'epochs': 5, 'n_features': 100, 'window': 2, 'word2vec': 'cbow', 'iteration': 1, 'accuracy': 6.0, 'time (s)': 1.13, 'empty_clusters': 0}


{'epochs': 5, 'n_features': 100, 'window': 2, 'word2vec': 'cbow', 'iteration': 2, 'accuracy': 18.0, 'time (s)': 0.9, 'empty_clusters': 0}


{'epochs': 5, 'n_features': 100, 'window': 2, 'word2vec': 'cbow', 'iteration': 3, 'accuracy': 10.0, 'time (s)': 1.06, 'empty_clusters': 0}


...

{'epochs': 5, 'n_features': 100, 'window': 2, 'word2vec': 'cbow', 'iteration': 7, 'accuracy': 16.0, 'time (s)': 0.95, 'empty_clusters': 0}


{'epochs': 5, 'n_features': 100, 'window': 2, 'word2vec': 'cbow', 'iteration': 8, 'accuracy': 12.0, 'time (s)': 1.11, 'empty_clusters': 0}


  0%|          | 0/6 [00:10<?, ?it/s]

{'epochs': 5, 'n_features': 100, 'window': 2, 'word2vec': 'cbow', 'iteration': 9, 'accuracy': 12.0, 'time (s)': 1.17, 'empty_clusters': 0}


KeyboardInterrupt: ignored

In [ ]:
hyperparameter_df = pd.DataFrame(hyperparameter_stats)
hyperparameter_df

In [ ]:
pd.set_option('display.max_rows', 500)

In [ ]:
# aggregate over the iterations per single Hyperparameter set
hyperparameter_stats_df = hyperparameter_df.groupby(['epochs', 'n_features', 'window', 'word2vec']).agg(accuracy_mean = ('accuracy', 'mean'), 
                                                                                                        accuracy_std = ('accuracy', 'std'),
                                                                                                        time_s = ('time (s)', 'mean'), 
                                                                                                        empty_clusters = ('empty_clusters', 'min')).reset_index()\
                                                                                                        .sort_values('accuracy_mean', ascending=False).reset_index(drop=True)

In [ ]:
hyperparameter_stats_df.to_csv('hyperparameter_stats_extensive.csv', index=False)

In [ ]:
hyperparameter_df.to_csv('hyperparameter_df_extensive.csv', index=False)

# Visualise results from Hyperparameter tuning

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mticks

In [ ]:
def lineplot_accuracy(df: pd.DataFrame, model: str = 'skip-gram', feature: str  = 'epochs'):
    best_accuracy = df.loc[df.word2vec == model, 'accuracy'].max()
    best_feature = df.loc[(df.word2vec == model) & (df.accuracy == best_accuracy), feature].iloc[0]
    max_iterations = df.iteration.max()

    fig, ax = plt.subplots(figsize=(6,6))
    plt.grid()
    plt.title(f'{model.title()} with {best_feature} {feature} of training results in highest accuracy')

    sns.lineplot(data=df.loc[df.word2vec == model], 
                x='iteration', y='accuracy', 
                hue=feature, ci=None, palette='deep', linewidth = 4)

    plt.xticks(np.arange(0, max_iterations+1))
    ax.yaxis.set_major_formatter(mticks.PercentFormatter(100))
    plt.ylim(0, 100)
    return ax

## Visualise `Skip-gram` and `CBOW` accuracy with change of `EPOCHS`

In [ ]:
ax1 = lineplot_accuracy(df=hyperparameter_df, model='skip-gram', feature = 'epochs')
plt.show()

In [ ]:
ax2 = lineplot_accuracy(df=hyperparameter_df, model='cbow', feature = 'epochs')
plt.show()

## Visualise `Skip-gram` and `CBOW` accuracy with change of `N Features`

There seems to be barely any difference between the feature vector sizes of 100, 200 and 300.

In [ ]:
ax2 = lineplot_accuracy(df=hyperparameter_df, model='skip-gram', feature = 'n_features')
plt.show()

In [ ]:
ax2 = lineplot_accuracy(df=hyperparameter_df, model='cbow', feature = 'n_features')
plt.show()

## Visualise `Skip-gram` and `CBOW` accuracy with change of `Window`

There seems to be barely any difference between the window sizes of 2 and 3.

In [ ]:
ax1 = lineplot_accuracy(df=hyperparameter_df, model='skip-gram', feature = 'window')
plt.show()

In [ ]:
ax2 = lineplot_accuracy(df=hyperparameter_df, model='cbow', feature = 'window')
plt.show()

# Manually observe best distributional semantics model

In [ ]:
window = 2
epochs = 30
d = 100
sg = 1


w2v_model = Word2Vec(window=window,
                    size=d,
                    sg=sg)
w2v_model.build_vocab(sentences)
w2v_model.train(sentences, 
                total_examples=w2v_model.corpus_count, 
                epochs=epochs)
w2v_model.init_sims(replace=True)

# transform the model vectors to a single matrix
X = create_feature_matrix(w2v_model)

kmeans = KMeans(n_clusters, algorithm='elkan').fit(X, y)

end = time()

# Calculate accuracy score
accuracy = calculate_accuracy(X, kmeans)

In [ ]:
accuracy

In [ ]:
w2v_model.most_similar('use')

In [ ]:
w2v_model.most_similar('ipod')

In [ ]:
w2v_model.most_similar('zen')